# Scrape Webpages in a Folder

In [10]:
import bs4
from bs4 import BeautifulSoup
import re
import json
import glob
from tqdm import tqdm
import pandas as pd

# Declare Parser Lambda

In [11]:
def parser_benefit(content):
    """
    Parse benefit text
        
    Returns:
        None if not found
    """
    benefit = content.find('h2', string=re.compile('公司福利'))
    
    if benefit is not None:
        benefit = benefit.find_next().text
        
        
    return benefit


parsers = {'company': lambda content: content.find('header', attrs={'class': ['header', 'mb-0']}).find('a', attrs={'class': 'cn'}).text,
          'title': lambda content: content.find('header', attrs={'class': ['header', 'mb-0']}).find('h1').find(text=True),
          'salary': lambda content: content.find('dd', attrs={'class': 'salary'}).find(text=True),
          'requirement': lambda content: content.find('h2', string='工作內容').find_parent('section').find('p').text,
          'requirement_others': lambda content: content.find('dt', string=re.compile('其他條件')).find_next().text,
          'address': lambda content: content.find('dd', attrs={'class': 'addr'}).find(text=True),
           'experience': lambda content: content.find('dt', string=re.compile('工作經歷')).find_next().text,
          'benefit': lambda content: parser_benefit(content)}

# Declare Parser Class

In [12]:
class HtmlParser():
    def __init__(self, parsers):
        self.parsers = parsers
        
        
    def parse(self, filename):
        
        with open(filename) as f:
            content = BeautifulSoup(f, "html.parser")
        
        jobObject = {}
        
        for key, parser in self.parsers.items():
            jobObject[key] = parser(content)
            
        return jobObject    
        
        
    def parse_pages(self, path_folder):
        """
        Args:
            path_folder (str): folder path of html files
            
        Returns:
            list of dict
        """
        jobs = []
        
        for f in tqdm(glob.glob(path_folder + "*.html")):
            jobObject = self.parse(f)
            jobs.append(jobObject)
            
        return jobs

# Declare Save Function

In [13]:
def save_job_objects(job_objects, filename, encoding='utf8'):
    """
    Args:
        job_objects (list of dict): list of jobs
    """
    with open(filename, 'w', encoding=encoding) as f:
        json.dump(job_objects, f, ensure_ascii=False)
    

# Main Function

In [28]:
path_folder='./dataset/'

# parse html pages in a folder
jobs = HtmlParser(parsers).parse_pages(path_folder)


100%|██████████| 996/996 [00:53<00:00, 18.65it/s]


In [29]:
print('Number of jobs={}\n'.format(len(jobs)))
print(jobs[:2])

Number of jobs=996

[{'company': '芯鼎科技股份有限公司', 'title': '\n                        系統應用工程師-台北(CD2320)                        ', 'salary': '待遇面議                                                                                    （', 'requirement': '- Driver development and fine tune of customized HW accelerator for OpenCV/OpenCL/OpenVX/CNN.- iCatchOS(RTOS) vision process framework development and maintain (SDK/BSP)- Integration and maintain of in-house design algorithms and performance fine tune.- Discuss/Design & co-work with customers for vision related requests.', 'requirement_others': '- A BS degree in Computer Engineering, Electrical Engineering, or Computer Science- At least 1-3 years of embedded software development experience (Senior – more than 5 years) - Familiar with C/C++ programming language, nice to have experience in Assembly language- Knowledge of Real-Time concepts and have RTOS experience (ThreadX , FreeRTOS, eCos, RTLinux, …)- Good understanding of CPU (e.g. ARM, MIPS 

In [30]:
filename_json='./dataset/jobs_104.json'

# save job (list of dictionary)
save_job_objects(jobs, filename_json)

# Inspect JSON file using Pandas

In [31]:
job_df = pd.read_json(filename_json)

In [35]:
len(job_df)

996

In [32]:
job_df.head()

,address,benefit,company,experience,requirement,requirement_others,salary,title
0,\n 台北市內...,\n福利健全 \n■ 優渥的薪資、獎金 \n◎ 端午節、中秋節、年終獎金。 \n◎ 定期依工...,芯鼎科技股份有限公司,不拘,- Driver development and fine tune of customiz...,"- A BS degree in Computer Engineering, Electri...",待遇面議 ...,\n 系統應用工程師-台北(CD2320) ...
1,\n 台北市大...,\n＊獎金福利＊1. 年節禮金2. 生日禮金3. 年終分紅4. 人才介紹獎金＊休假福利＊1....,銓鴻資訊有限公司,1年以上,1. 研究市場數據和交易數據，並進行交易策略研發。 \n2. 開發模型和工具，監控分析交易程...,將先以電子履歷篩選增加效率，請注意以下事項：\n■ 履歷請詳述相關經驗並請提供 Github...,"年薪 1,000,000~2,500,000元 ...",\n 計量交易員 Quantitative T...
2,\n 新北市新...,\n《薪酬福利＠HTC》我們極力落實利潤與員工分享的精神，透過多元的薪酬組合與完善的福利方案...,宏達電 HTC Corporation_宏達國際電子股份有限公司,不拘,Join the creative thinkers at HTC Healthcare. ...,Requirement:-MS or PhD in computer science or ...,待遇面議 ...,\n (RD S/W) DeepQ - Com...
3,\n 桃園市中...,\n正職/全職人員之福利與訓練制度：\n\n【薪酬福利】\n-每年依市場薪資水準評估薪資調整...,OK超商_來來超商股份有限公司,不拘,OK超商認為，用心經營的企業，首重人才培訓。\n\n因此，我們投入了大量的訓練資源，積極培育...,1.具服務熱忱，科系、經驗不拘，須輪班。\n2.須自備交通工具及電腦網路設備(透過e-Lea...,"月薪 31,000~33,000元 ...",\n 儲備幹部(大桃園地區) ...
4,\n 新竹縣竹...,\n【工作福利】彈性上下班。 \n【休假福利】週休二日、國定假日、婚假、產假、陪產假、特休假...,神亞科技股份有限公司,3年以上,影像相關之深度學習演算法研究與開發,1. 有開發image processing相關應用經驗\n2. 熟悉深度學習相關算法（e....,待遇面議 ...,\n AI 工程師 ...


In [27]:
job_df[-10:]

,address,benefit,company,experience,requirement,requirement_others,salary,title
897,\n 新北市板...,\n(1)保險：享有勞保、健保及6%勞退金提撥\n(2)員工福利：\n＊康樂福利：國內外旅遊...,香繼光集團繼光香香雞_香繼光股份有限公司,不拘,1.親切的服務與接待\n2.為客人介紹餐點、點菜、出餐\n3.櫃台服務，收銀、結帳\n4.依...,1.具備機車駕照者優先錄取\n\n香繼光集團有完善的教育訓練和晉升管道，歡迎有服務熱誠且想在...,時薪 160~180元/小時 ...,\n [環球板橋店]-兼職人員-時薪160元起...
898,\n 新竹市篤...,\n<完善的在職訓練>\n公司全力支持同仁追求新知與學習新技術，協助同仁取得全球最新的技術文...,聯詠科技股份有限公司,不拘,"1.\t熟悉Linux系統, 具C/C++/Python程式語言設計經驗 \n2.\t熟悉M...",擅長工具：C、C++、Python、TensorFlow、Caffe、Keras,待遇面議 ...,\n AI語音系統軟體開發工程師 (SoC_L...
899,\n 台中市北...,\n尚未提供\n,澳洲商艾萊門窗股份有限公司台灣分公司,5年以上,Report to Global HQ with leadership position a...,"Job Requirements (include, but are not limited...",待遇面議 ...,\n Senior Manufacturing...
900,\n 台北市南...,\n〔薪資福利〕 \n- 獎金(分紅/年終) \n- 勞保、健保、退休金提撥 \n- ...,願境網訊股份有限公司_KKBOX,不拘,About KKStreamKKStream is a KKBOX branch estab...,Requirement* You have the ability to condense ...,待遇面議 ...,\n [KKStream] Data Anal...
901,\n 台南市六...,\n1.勞、健保 2.三節禮金 3.免費供午膳 4.勞退6%提撥 5.免費...,世洲機械廠股份有限公司,1年以上,-專科以上機械相關科系畢業\n-熟AUTOCAD、EXCEL、WORD\n-具工作經驗者,未填寫,"月薪 28,000~35,000元 ...",\n 機械繪圖人員 ...
902,\n 新竹縣竹...,\n這是一個為研究菁英打造的研發環境，我們的訴求是讓研究人員能在此構築人生理想及技術成就，讓...,工研院 _財團法人工業技術研究院,2年以上,•Strong knowledge and experience with Machine ...,科系:電機工程、資訊工程等相關科系。工作經驗:1.工作2年以上尤佳2.曾參加過國內外競賽得獎...,待遇面議 ...,\n 工研院服科中心_AI軟體研發工程師(U4...
903,\n 台北市大...,\n1. 特休全年十天起跳，最多三十天\n2. 事假、病假、喪假、婚假、產假、陪產假\n3....,樺鼎商業資訊股份有限公司,3年以上,1. SW Integration Development & Maintain\n2. I...,1. at least 3 years in project base SW develop...,待遇面議 ...,\n 系統分析規劃顧問 ...
904,\n 新北市中...,\n◆ 獎金/禮品類\n1.年終獎金\n2.三節獎金/禮品\n3.佳節禮品\n\n◆ 保險類...,文曄集團_文曄科技股份有限公司,3年以上,【誰能成為WT菁英】\n1.國內外大學或研究所之資訊工程/資管等相關系所畢業。\n2.配合公...,1.熟資料庫語法\n2.具ETL設計經驗\n3.Oracle 資料庫開發經驗\n4.具 Ta...,待遇面議 ...,\n 商業智慧(BI)分析師(IT) ...
905,\n 台北市信...,\n本公司正職員工享有以下福利：\n\n● 年終獎金\n● 年度健檢\n● 三節禮金\n● ...,17直播_英屬維京群島商麻吉一七股份有限公司臺灣分公司,2年以上,- QA experiences for mobile/web platforms with...,- 2 - 5 years of related experience.\n- Better...,待遇面議 ...,\n QA Engineer 軟體測試工程師 ...
906,\n 台北市信...,\n我們提供優於法令規定及整體金融產業的福利措施如下:\n◎優於勞基法規定之休假制度\n\n...,(總公司)南山人壽保險股份有限公司,3年以上,● 異質資料庫數據整合，建置資料倉儲與管理 ● 數據分析、應用之相關程式開發 ● 資料庫管理...,● 熟T-SQL語法及Stored Procedure開發● 熟MS SQL Server ...,待遇面議 ...,\n 金融數據工程師 ...
